<a href="https://colab.research.google.com/github/koleshjr/collab_projects/blob/main/Ensembling_Voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENSEMBLING_VOTING

In [ ]:
#Import some libraries
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import plotly.express as px
from pylab import rcParams
import seaborn as sb
sb.set_style('darkgrid')
rcParams['figure.figsize'] = 8,8
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
# from imblearn.over_sampling import SMOTE
import lightgbm as lgb
import xgboost as xgb
import catboost as catt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss,roc_auc_score,accuracy_score

# from imblearn.over_sampling import RandomOverSampler
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,RepeatedStratifiedKFold, StratifiedKFold, KFold, cross_val_score

pd.options.display.max_rows = 2200
pd.options.display.max_columns = 2200

In [ ]:
#import data
path = "./"
train = pd.read_csv(path+'alvintrain_base.csv')
test=  pd.read_csv(path+'alvintest_base.csv')
submission = pd.read_csv(path+'SampleSubmission.csv')

In [ ]:
train_df = train.copy()
test_df = test.copy()
train_df.shape,test_df.shape

((373, 48), (558, 48))

In [ ]:
main_cols = train_df.columns.difference(['USER_ID','MERCHANT_NAME_grouped','purchase_merch_mean','Transaction_ID','PURCHASED_ATyear','PURCHASE_VALUE','PURCHASED_ATmonth','PURCHASED_ATdayofweek','MD_PURCHASED_ATMERCHANT_CATEGORIZED_AT','MERCHANT_CATEGORIZED_AS','MERCHANT_CATEGORIZED_ATmonth','MERCHANT_CATEGORIZED_ATdayofweek','YD_PURCHASED_ATMERCHANT_CATEGORIZED_AT',                                         
'bins_income_min', 'bins_income_sum', 'bins_purchase_merch_count',
       'bins_purchase_merch_max',
                                        
                                        
                                        ])
X = train_df[main_cols]
y = train_df.MERCHANT_CATEGORIZED_AS
test = test_df[main_cols]

In [ ]:
from imblearn.over_sampling import  SMOTE
oversample = SMOTE(k_neighbors=1)
X, y = oversample.fit_resample(X, y)

In [ ]:
X.head()

,DD_PURCHASED_ATMERCHANT_CATEGORIZED_AT,IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_False,IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY_True,MERCHANT_CATEGORIZED_ATyear,MERCHANT_NAME,MERCHANT_NAME_bins,USER_GENDER_Female,USER_GENDER_Male,USER_HOUSEHOLD,USER_INCOME,bins_income_max,bins_income_mean,bins_purchase_merch_mean,bins_purchase_merch_min,bins_purchase_merch_sum,count_DD_PURCHASED_ATMERCHANT_CATEGORIZED_AT,count_MERCHANT_NAME,count_PURCHASE_VALUE,count_USER_HOUSEHOLD,count_USER_INCOME,income_max,income_mean,income_min,income_range,income_sum,price_range,prod_PURCHASE_INCOME,purchase_merch_count,purchase_merch_max,purchase_merch_min,purchase_merch_sum
0,0,1,0,1,413,9.0,0,1,3,10000,1000000,187038.961039,1419.240260,26,218563,72,1,4,240,188,10000.0,10000.000000,10000.0,3,70000.0,3,0.170000,7.0,2720.0,100.0,6220.0
1,-1,1,0,0,299,3.0,1,0,4,90000,1000000,284849.112426,2618.263314,50,884973,18,2,78,91,90,0.0,0.000000,0.0,0,0.0,3,0.001111,0.0,0.0,0.0,0.0
2,6,1,0,1,348,1.0,0,1,3,10000,1000000,148669.673055,485.801578,1,861812,18,27,19,240,188,350000.0,56804.794521,8000.0,3,49761000.0,3,0.002000,876.0,1000.0,2.0,36731.0
3,-64,1,0,1,434,8.0,0,1,5,252000,1000000,142526.205451,3067.886792,6,2926764,4,2,1,189,122,0.0,0.000000,0.0,0,0.0,3,0.015079,0.0,0.0,0.0,0.0
4,0,1,0,1,244,5.0,0,1,3,10000,1000000,206164.197531,1814.962346,2,5880478,72,4,18,240,188,30000.0,14000.000000,10000.0,3,140000.0,3,0.006000,10.0,150.0,30.0,600.0


In [ ]:
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier
import time

seed = 21

rnf_params = {'max_depth': 12, 'max_features':'log2', 'n_estimators': 1100, 'random_state': seed}

xgb_params = {"eta":0.1, 'n_estimators': 50, 'random_state':seed, 'objective':'mlogloss',
              "eval_metric":'logloss', "subsample" : 1.0, "colsample_bytree" : .8}

cat_params = {"iterations":10000, "learning_rate":0.09, "verbose":100, "random_state":seed,
             'use_best_model':True, 'early_stopping_rounds':100}

lgb_params = {'learning_rate':0.01, 'n_estimators':10000, 'random_state':seed,
              'early_stopping_round':100, 'verbose':-1, 'subsample':.8}

 

def get_models():
    _models = [
            CatBoostClassifier(**cat_params),   
            LGBMClassifier(**lgb_params),
            XGBClassifier(**xgb_params),
            # RandomForestClassifier(**rnf_params),
            # BaggingClassifier(GradientBoostingClassifier(max_features='log2'), n_jobs=-1, max_features=0.8, random_state=seed)
    ]
    return _models

class My_model:
    def __init__(self, models):
        self.models = models

    def fit_eval_pred(self, X, y, eval_set, val, test):
        results_eval = []
        results_test = []
        for i, model in enumerate(self.models):
            print(f"[Training]............................... Model_{i+1}")
            st = time.time()
            try:
                model.fit(X, y, eval_set=eval_set, verbose=False)
            except Exception as e:
                X = X.fillna(11)
                val = val.fillna(11)
                test = test.fillna(11)
                model.fit(X, y)
            print(f"[Prediction]............................. Model_{i+1}")
            results_eval.append(model.predict_proba(val))
            results_test.append(model.predict_proba(test))
            print(f"Time {time.time() - st :.2f}sec")

        results_eval = np.mean(results_eval, axis=0)
        results_test = np.mean(results_test, axis=0)

        return results_eval, results_test

In [ ]:
results = []
ns = 2
tot = []


skf = StratifiedKFold(n_splits=ns)
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(100*"-")
    print(f"Fold-{fold+1}")
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]


    main_model = My_model(get_models())

    pred, T_pred = main_model.fit_eval_pred(X_train, y_train, [(X_test.values, y_test)], X_test, test)

    try:
      loss = log_loss(y_test, pred)
      print(f"log loss = {loss}")
      tot += [loss]
    except Exception as e:
      print(e)


    results.append(T_pred)
    print(100*"-")
    print("\n\n")

print(f"Average log loss = {sum(tot) / len(tot)}")

----------------------------------------------------------------------------------------------------
Fold-1
[Training]............................... Model_1
[Prediction]............................. Model_1
Time 42.80sec
[Training]............................... Model_2
[Prediction]............................. Model_2
Time 6.95sec
[Training]............................... Model_3
[Prediction]............................. Model_3
Time 0.53sec
log loss = 0.47303730244535835
----------------------------------------------------------------------------------------------------



----------------------------------------------------------------------------------------------------
Fold-2
[Training]............................... Model_1
[Prediction]............................. Model_1
Time 71.98sec
[Training]............................... Model_2
[Prediction]............................. Model_2
Time 6.13sec
[Training]............................... Model_3
[Prediction]....................

In [ ]:
from sklearn.ensemble import BaggingClassifier

seed = 21

rnf_params = {'max_depth': 16, 'max_features':'log2', 'n_estimators': 1000, 'random_state': seed}
lgb_params = {"learning_rate":0.009, "n_estimators":1000, 'random_state':seed, 'sub_sample' : 0.7,'colsample_bytree' : 0.6}

xgb_params = {"eta":0.3, 'n_estimators': 200, 'random_state':seed}
cat_params = {"iterations":1000, "learning_rate":0.01, "verbose":0, "random_state":seed}

            
def model():
    models = [
            ('lgb', LGBMClassifier(**lgb_params)),
            ('rnd', RandomForestClassifier(**rnf_params)),
            ('xgb', XGBClassifier(**xgb_params)),
            ('cat', CatBoostClassifier(**cat_params)),
            ('bag', BaggingClassifier(RandomForestClassifier(), n_jobs=-1, max_features=0.9, random_state=seed))
    ]
    return VotingClassifier(estimators=models, voting='soft', flatten_transform=True, verbose=1)

In [ ]:
clf = model()
clf.fit(X, y)
pred = clf.predict_proba(X)
log_loss(y, pred)

[LightGBM] [Warning] Unknown parameter: sub_sample
[Voting] ...................... (1 of 5) Processing lgb, total=  11.5s
[Voting] ...................... (2 of 5) Processing rnd, total=   2.8s
[13:28:57] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[Voting] ...................... (3 of 5) Processing xgb, total=   1.7s
[Voting] ...................... (4 of 5) Processing cat, total=  36.5s
[Voting] ...................... (5 of 5) Processing bag, total=   5.1s


0.10119480162651515

### No smote 0.22257032258035764 - 1.264679281721714
### Smote 0.1031406001692244 - 1.212545960479536
### Smote 0.10119480162651515 - 1.193047116075098

In [ ]:
# Get the predicted result for the test Data
predictions = clf.predict_proba(test)
predictions.shape

(558, 13)

In [ ]:
preds = predictions
preds_df = pd.DataFrame(preds, columns=['Bills & Fees',
'Data & WiFi',
'Education',
'Emergency fund',
'Family & Friends',
'Going out',
'Groceries',
'Health',
'Loan Repayment',
'Miscellaneous',
'Rent / Mortgage',
'Shopping',
'Transport & Fuel',
])
preds_df = pd.concat(objs=[test_df['Transaction_ID'], preds_df], axis="columns")
preds_df.head()

,Transaction_ID,Bills & Fees,Data & WiFi,Education,Emergency fund,Family & Friends,Going out,Groceries,Health,Loan Repayment,Miscellaneous,Rent / Mortgage,Shopping,Transport & Fuel
0,ID_00x9h2yx,0.460936,0.014046,0.008133,0.180784,0.200472,0.015661,0.020731,0.003730,0.035725,0.014544,0.004044,0.035028,0.006166
1,ID_01db594f,0.048886,0.013235,0.002207,0.031659,0.007228,0.782010,0.022590,0.004711,0.012435,0.005898,0.002137,0.056237,0.010769
2,ID_07pm99no,0.015698,0.001230,0.000492,0.002692,0.013897,0.077508,0.165049,0.000835,0.001270,0.699102,0.000358,0.002962,0.018909
3,ID_09jryhkd,0.179360,0.009885,0.007760,0.043059,0.038702,0.140496,0.025137,0.006792,0.042298,0.331041,0.045937,0.027836,0.101698
4,ID_0ccv924c,0.500787,0.009822,0.002911,0.214531,0.025287,0.085825,0.028127,0.003520,0.035178,0.008866,0.039228,0.037703,0.008216


In [ ]:
preds_df.to_csv('voting.csv', index=False)